In [21]:
#Create X_anomalies
import os
import numpy as np
import joblib
import tensorflow as tf

# === Paths ===
BASE_DIR = "/content/drive/MyDrive/spacecraft_anomaly_project"
PROCESSED_DIR = os.path.join(BASE_DIR, "data", "processed")
MODEL_DIR = os.path.join(BASE_DIR, "models/autoencoder")

# === Load Scaler ===
scaler_path = os.path.join(PROCESSED_DIR, "scaler.pkl")
scaler = joblib.load(scaler_path)
print("✅ Scaler loaded.")

# === Load X_full (scaled data) ===
x_full_path = os.path.join(PROCESSED_DIR, "X_full.npy")
X_full = np.load(x_full_path)
print(f"✅ X_full loaded: shape = {X_full.shape}")

# === Reshape if needed for LSTM ===
# Assuming the shape for training was (samples, timesteps, features)
# Change 'timesteps' to match what you used during training
TIMESTEPS = 30
n_features = X_full.shape[1]

X_full_seq = []
for i in range(len(X_full) - TIMESTEPS + 1):
    X_full_seq.append(X_full[i:i+TIMESTEPS])
X_full_seq = np.array(X_full_seq)
print(f"✅ Reshaped for LSTM: {X_full_seq.shape}")

# === Load trained autoencoder ===
autoencoder_path = os.path.join(MODEL_DIR, "lstm_autoencoder.h5")
autoencoder = tf.keras.models.load_model(autoencoder_path)
print("✅ Autoencoder loaded.")

# === Compute reconstruction error ===
X_full_pred = autoencoder.predict(X_full_seq)
recon_errors = np.mean(np.square(X_full_seq - X_full_pred), axis=(1, 2))

# === Set anomaly threshold ===
# Option 1: Fixed threshold
threshold = np.percentile(recon_errors, 95)  # top 5% errors = anomalies
# Option 2: Use previously saved threshold if available
# threshold = joblib.load(os.path.join(PROCESSED_DIR, "threshold.pkl"))

print(f"📏 Using threshold: {threshold}")

# === Detect anomalies ===
anomaly_flags = recon_errors > threshold
print(f"🚨 Detected {np.sum(anomaly_flags)} anomalies out of {len(anomaly_flags)} sequences.")

# === Extract anomaly windows ===
X_anomalies = X_full_seq[anomaly_flags]
print(f"✅ X_anomalies shape: {X_anomalies.shape}")

# === Save anomalies for TTI model ===
np.save(os.path.join(PROCESSED_DIR, "X_anomalies.npy"), X_anomalies)
print("💾 X_anomalies.npy saved.")

✅ Scaler loaded.
✅ X_full loaded: shape = (2123, 21)
✅ Reshaped for LSTM: (2094, 30, 21)


✅ Autoencoder loaded.
66/66 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step
📏 Using threshold: 3.097266131730625
🚨 Detected 105 anomalies out of 2094 sequences.
✅ X_anomalies shape: (105, 30, 21)
💾 X_anomalies.npy saved.


In [31]:
import os
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler

# Paths
BASE_DIR = "/content/drive/MyDrive/spacecraft_anomaly_project"
PROCESSED_DIR = os.path.join(BASE_DIR, "data/processed")
MODELS_DIR = os.path.join(BASE_DIR, "models")
os.makedirs(MODELS_DIR, exist_ok=True)

# Load the full scaled data
X_full = np.load(os.path.join(PROCESSED_DIR, 'X_full.npy'))
print(f"✅ X_full loaded: shape = {X_full.shape}")

# Fit scaler for TTI model (no flattening needed)
tti_scaler = StandardScaler()
tti_scaler.fit(X_full)

# Save the TTI scaler
joblib.dump(tti_scaler, os.path.join(MODELS_DIR, 'tti_scaler.pkl'))
print("✅ TTI scaler saved at:", os.path.join(MODELS_DIR, 'tti_scaler.pkl'))

✅ X_full loaded: shape = (2123, 21)
✅ TTI scaler saved at: /content/drive/MyDrive/spacecraft_anomaly_project/models/tti_scaler.pkl


In [52]:
import os
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# ==============================
# CONFIG
# ==============================
BASE_DIR = "/content/drive/MyDrive/spacecraft_anomaly_project"
MODELS_DIR = os.path.join(BASE_DIR, "models")
DATA_DIR = os.path.join(BASE_DIR, "data/processed")

os.makedirs(MODELS_DIR, exist_ok=True)

# ==============================
# LOAD ANOMALIES
# ==============================
X_anomalies = np.load(os.path.join(DATA_DIR, "X_anomalies.npy"))
print(f"✅ Loaded anomalies: {X_anomalies.shape}")  # (samples, timesteps, features)

# Optional: Load reconstruction errors
reconstruction_errors_path = os.path.join(DATA_DIR, "reconstruction_errors.npy")
if os.path.exists(reconstruction_errors_path):
    reconstruction_errors = np.load(reconstruction_errors_path)
    print("✅ Loaded reconstruction errors for severity-weighted TTI generation.")
else:
    reconstruction_errors = None
    print("⚠️ No reconstruction errors found, using random TTI generation.")

# ==============================
# GENERATE SYNTHETIC TTI LABELS
# ==============================
min_tti = 300      # 5 minutes
max_tti = 7200     # 2 hours

if reconstruction_errors is not None:
    errors = (reconstruction_errors - reconstruction_errors.min()) / (reconstruction_errors.max() - reconstruction_errors.min() + 1e-8)
    y_tti_synthetic = max_tti - (errors * (max_tti - min_tti)).astype(int)
else:
    y_tti_synthetic = np.random.randint(min_tti, max_tti + 1, size=X_anomalies.shape[0])

print(f"✅ Synthetic TTI labels generated. Example: {y_tti_synthetic[:10]}")

# ==============================
# SCALE FEATURES
# ==============================
# Scale per-feature across all timesteps
n_samples, timesteps, n_features = X_anomalies.shape
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_anomalies.reshape(-1, n_features)).reshape(n_samples, timesteps, n_features)

# ==============================
# BUILD LSTM REGRESSOR
# ==============================
model = Sequential([
    LSTM(64, activation='tanh', input_shape=(timesteps, n_features), return_sequences=False),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1)  # Output TTI in seconds
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# ==============================
# TRAIN MODEL
# ==============================
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint(
    os.path.join(MODELS_DIR, "tti_lstm_regressor.keras"),
    save_best_only=True,
    monitor='val_loss'
)

history = model.fit(
    X_scaled, y_tti_synthetic,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    callbacks=[early_stop, checkpoint],
    verbose=1
)

print("✅ LSTM regressor trained and saved.")

# ==============================
# SAVE SCALER
# ==============================
with open(os.path.join(MODELS_DIR, "tti_scaler.pkl"), "wb") as f:
    pickle.dump(scaler, f)

✅ Loaded anomalies: (105, 30, 21)
⚠️ No reconstruction errors found, using random TTI generation.
✅ Synthetic TTI labels generated. Example: [2311 1114 4113 6574 6391 3296  821 6903 2222  786]
Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 334ms/step - loss: 19162016.0000 - mae: 3806.3975 - val_loss: 16713469.0000 - val_mae: 3593.8535
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 107ms/step - loss: 17813620.0000 - mae: 3630.1555 - val_loss: 16711421.0000 - val_mae: 3593.5696
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - loss: 17049676.0000 - mae: 3512.1584 - val_loss: 16709051.0000 - val_mae: 3593.2410
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step - loss: 17856628.0000 - mae: 3604.9946 - val_loss: 16705926.0000 - val_mae: 3592.8059
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - loss: 18344324.0000 - mae: 3756.4771 - val_loss: 16701882.0000 - val_mae: 3592.2415
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - loss: 19523624.0000 - mae: 3839.4429 - val_loss: 16696431.0000 - val_mae: 3591.4829
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 18682262.0000 - mae: 3708.3740 - val_loss: 16690165.0000 - val_mae: 3590.6116
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 189

In [56]:
# ===== TTI regressor prediction (LSTM-ready) =====
import os
import numpy as np
import joblib
from tensorflow import keras

# ===== PATHS =====
BASE_DIR = "/content/drive/MyDrive/spacecraft_anomaly_project"  # adjust only if needed
PROCESSED_DIR = os.path.join(BASE_DIR, "data", "processed")
MODELS_DIR = os.path.join(BASE_DIR, "models")

# ===== LOAD ANOMALIES =====
X_anomalies = np.load(os.path.join(PROCESSED_DIR, 'X_anomalies.npy'))
print(f"✅ X_anomalies loaded: shape = {X_anomalies.shape}")

# ===== LOAD TTI SCALER (joblib) =====
tti_scaler = joblib.load(os.path.join(MODELS_DIR, 'tti_scaler.pkl'))
print("✅ TTI scaler loaded with joblib.")

# ===== SCALE ANOMALIES PER TIMESTEP =====
n_samples, timesteps, n_features = X_anomalies.shape
# use float dtype for scaled array
X_anomalies_scaled = np.zeros((n_samples, timesteps, n_features), dtype=np.float32)

for t in range(timesteps):
    # transform each timestep's feature vector (shape: n_samples x n_features)
    X_anomalies_scaled[:, t, :] = tti_scaler.transform(X_anomalies[:, t, :])

print("✅ Anomalies scaled (per-timestep). Shape:", X_anomalies_scaled.shape)

# ===== LOAD TTI LSTM MODEL =====
tti_model = keras.models.load_model(os.path.join(MODELS_DIR, "tti_lstm_regressor.keras"))
print("✅ TTI model loaded.")

# ===== PREDICT TTI (keep 3D input for LSTM) =====
# model expects shape (batch, timesteps, features)
tti_predictions = tti_model.predict(X_anomalies_scaled, verbose=0)
tti_predictions = np.array(tti_predictions).reshape(-1)  # (n_samples,)
print("✅ TTI predictions complete. Example predictions:", tti_predictions[:10])

# ===== SORT ANOMALIES BY SHORTEST TTI =====
sorted_indices = np.argsort(tti_predictions)  # ascending: smallest (most urgent) first
X_anomalies_prioritized = X_anomalies[sorted_indices]
tti_prioritized = tti_predictions[sorted_indices]

print("\n📊 Top prioritized anomalies (shortest TTI first):")
for i, tti in enumerate(tti_prioritized[:10], start=1):
    print(f"{i}. TTI = {tti:.2f} units")

# ===== SAVE PRIORITIZED RESULTS =====
np.save(os.path.join(PROCESSED_DIR, 'X_anomalies_prioritized.npy'), X_anomalies_prioritized)
np.save(os.path.join(PROCESSED_DIR, 'tti_prioritized.npy'), tti_prioritized)
print("💾 Saved prioritized anomalies & TTI predictions to processed/")

# ===== Optional: produce a small CSV for quick inspection =====
import pandas as pd
top_n = min(50, len(tti_prioritized))
df_out = pd.DataFrame({
    'anomaly_idx': sorted_indices[:top_n],
    'predicted_tti': tti_prioritized[:top_n]
})
df_out.to_csv(os.path.join(PROCESSED_DIR, 'tti_prioritized_top50.csv'), index=False)
print(f"💾 Saved top-{top_n} prioritized list as CSV.")

✅ X_anomalies loaded: shape = (105, 30, 21)
✅ TTI scaler loaded with joblib.
✅ Anomalies scaled (per-timestep). Shape: (105, 30, 21)
✅ TTI model loaded.
✅ TTI predictions complete. Example predictions: [115.39368  115.47932  115.496605 115.45666  115.489044 115.46363
 115.428314 115.45606  115.5001   115.46639 ]

📊 Top prioritized anomalies (shortest TTI first):
1. TTI = 115.20 units
2. TTI = 115.28 units
3. TTI = 115.32 units
4. TTI = 115.36 units
5. TTI = 115.38 units
6. TTI = 115.38 units
7. TTI = 115.38 units
8. TTI = 115.38 units
9. TTI = 115.38 units
10. TTI = 115.39 units
💾 Saved prioritized anomalies & TTI predictions to processed/
💾 Saved top-50 prioritized list as CSV.


In [37]:
import shutil
import os

MODELS_DIR = "/content/drive/MyDrive/spacecraft_anomaly_project/models"

shutil.copy(
    os.path.join(MODELS_DIR, 'scaler.pkl'),
    os.path.join(MODELS_DIR, 'tti_scaler.pkl')
)
print("✅ Copied scaler.pkl to tti_scaler.pkl")


✅ Copied scaler.pkl to tti_scaler.pkl
